# Make figures

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import glob
import sys
from shapely import wkt
import seaborn as sns
import string
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression
import ast
from matplotlib.patches import Rectangle
import matplotlib

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

# scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'
figures_out_path = os.path.join(base_path, 'figures')
aois_fn = os.path.join(scm_path, 'compiled_data', 'all_aois_climate_cluster.shp')
eras_fn = os.path.join(scm_path, 'compiled_data', 'all_era_data.csv')
clusters_fn = os.path.join(scm_path, 'compiled_data', 'climate_clusters.csv')
snowlines_fn = os.path.join(scm_path, 'compiled_data', 'all_snowlines.csv')

## Define some colormaps

In [ ]:
# Climate clusters
cluster_cmap_dict = {'Oceanic': '#1f78b4', 
                     'Continental': '#e31a1c',
                     'Transitional-Continental': '#fb9a99',
                     'Transitional-Temperate': '#b2df8a',
                     'Temperate': '#33a02c'}


## AAR time series

In [ ]:
# -----Load AOIs
aois = gpd.read_file(aois_fn)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)
print('All glacier boundaries loaded from file.')

# -----Load snowlines
snowlines = pd.read_csv(snowlines_fn)
snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
print('All snowlines loaded from file.')

In [ ]:
# -----Add Year and WOY columns to snowlines
snowlines['Year'] = snowlines['datetime'].dt.isocalendar().year
snowlines['WOY'] = snowlines['datetime'].dt.isocalendar().week

# -----Add subregion and climate cluster columns to snowlines
snowlines[['Subregion', 'color']] = '', '', 
for o1, o2 in tqdm(aois[['O1Region', 'O2Region']].drop_duplicates().values):
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    aois_subregion = aois.loc[(aois['O1Region']==o1) & (aois['O2Region']==o2)]
    site_names = aois_subregion['RGIId'].drop_duplicates().values
    for site_name in site_names:
        snowlines.loc[snowlines['site_name']==site_name, 'O1Region'] = o1
        snowlines.loc[snowlines['site_name']==site_name, 'O2Region'] = o2
        snowlines.loc[snowlines['site_name']==site_name, 'Subregion'] = subregion_name
        snowlines.loc[snowlines['site_name']==site_name, 'color'] = color
snowlines[['cluster', 'clustName']] = '', '', 
for site_name in (climate_clusters['site_name'].drop_duplicates().values):
    climate_clusters_site = climate_clusters.loc[climate_clusters['site_name']==site_name]
    snowlines.loc[snowlines['site_name']==site_name, 'cluster'] = climate_clusters_site['cluster'].values[0]
    snowlines.loc[snowlines['site_name']==site_name, 'clustName'] = climate_clusters_site['clustName'].values[0]
snowlines = snowlines.loc[snowlines['cluster']!='']
snowlines.reset_index(drop=True, inplace=True)

### By subregion

In [ ]:
# -----Set up figure
fig, ax = plt.subplots(10, 2, figsize=(12, 40), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

q1, q3 = 0.25, 0.75

# -----Iterate over subregions
o1o2s = snowlines[['O1Region', 'O2Region']].drop_duplicates().dropna().sort_values(by=['O1Region', 'O2Region']).values
ymin, ymax = -0.1, 1.1
for i, (o1, o2) in enumerate(o1o2s):
    # subset snowlines to subregion
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]
    if len(snowlines_subregion) < 1:
        continue
    subregion_name, color = snowlines_subregion[['Subregion', 'color']].values[0]
    if color=='':
        color='k'
    # calculate moving median over time
    ts_gb = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR']
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    ts['AAR_Q1'] = ts_gb.quantile(q1).reset_index()['AAR']
    ts['AAR_Q3'] = ts_gb.quantile(q3).reset_index()['AAR']
    # calculate moving median for all years stacked
    weekly_gb = snowlines_subregion.groupby(by='WOY')['AAR']
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    weekly['AAR_Q1'] = weekly_gb.quantile(q1).reset_index()['AAR']
    weekly['AAR_Q3'] = weekly_gb.quantile(q3).reset_index()['AAR']
    weekly['WOY'] = weekly['WOY'].values.astype(float)
    # plot
    # add grey boxes where observations don't exist
    for year in ts['Year'].values:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin), width=t2-t1, height=ymax-ymin, facecolor='#d9d9d9', edgecolor='None'))
        ts_year = ts.loc[ts['Year']==year]
        ax[i*2].plot(ts_year['date'], ts_year['AAR_median'], 'o', color=color, linewidth=0.5, 
                     markerfacecolor=color, markeredgecolor='w', markersize=5)
    ax[i*2].set_ylim(ymin, ymax)
    ax[i*2].grid()
    ax[i*2].set_ylabel('AAR')
    N = len(snowlines_subregion['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{subregion_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[(i*2)+1].set_ylim(ymin, ymax)
    ax[(i*2)+1].grid()
    ax[(i*2)+1].fill_between(weekly['WOY'], weekly['AAR_Q1'], weekly['AAR_Q3'], color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly['AAR_median'], '-', linewidth=2, color=color)
    # change week numbers to month labels
    ax[(i*2)+1].set_xticks([22, 31, 40])
    ax[(i*2)+1].set_xticklabels(['June', 'Aug', 'Oct'])

fig.subplots_adjust(hspace=0.3)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_subregions.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### By climate cluster

In [ ]:
# -----Set up figure
n_clusters = len(snowlines['cluster'].drop_duplicates().values)
fig, ax = plt.subplots(n_clusters, 2, figsize=(12, 4*n_clusters), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

q1, q3 = 0.25, 0.75

# -----Iterate over subregions
ymin, ymax = -0.1, 1.1
for i, cluster_name in enumerate(['Oceanic', 
                                  'Continental', 
                                  'Transitional-Continental', 
                                  'Transitional-Temperate', 
                                  'Temperate']):
    # subset snowlines to cluster
    snowlines_cluster = snowlines.loc[snowlines['clustName']==cluster_name]
    if len(snowlines_cluster) < 1:
        continue
    # grab cluster name
    color = cluster_cmap_dict[cluster_name]
    # calculate moving median over time
    ts_gb = snowlines_cluster.groupby(by=['Year', 'WOY'])['AAR']
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    ts['AAR_Q1'] = ts_gb.quantile(q1).reset_index()['AAR']
    ts['AAR_Q3'] = ts_gb.quantile(q3).reset_index()['AAR']
    # calculate moving median for all years stacked
    weekly_gb = snowlines_cluster.groupby(by='WOY')['AAR']
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    weekly['AAR_Q1'] = weekly_gb.quantile(q1).reset_index()['AAR']
    weekly['AAR_Q3'] = weekly_gb.quantile(q3).reset_index()['AAR']
    weekly['WOY'] = weekly['WOY'].values.astype(float)
    # plot
    # add grey boxes where observations don't exist
    for year in ts['Year'].values:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin), width=t2-t1, height=ymax-ymin, facecolor='#d9d9d9', edgecolor='None'))
        ts_year = ts.loc[ts['Year']==year]
        ax[i*2].plot(ts_year['date'], ts_year['AAR_median'], 'o', color=color, linewidth=0.5, 
                     markerfacecolor=color, markeredgecolor='w', markersize=5)
    ax[i*2].set_ylim(ymin, ymax)
    ax[i*2].grid()
    ax[i*2].set_ylabel('AAR')
    N = len(snowlines_cluster['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{cluster_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[(i*2)+1].set_ylim(ymin, ymax)
    ax[(i*2)+1].grid()
    ax[(i*2)+1].fill_between(weekly['WOY'], weekly['AAR_Q1'], weekly['AAR_Q3'], color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly['AAR_median'], '-', linewidth=2, color=color)
    # change week numbers to month labels
    ax[(i*2)+1].set_xticks([22, 31, 40])
    ax[(i*2)+1].set_xticklabels(['June', 'Aug', 'Oct'])

fig.subplots_adjust(hspace=0.3)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_clusters.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### Annual trends stacked

In [ ]:
# -----Set up figure
fig, ax = plt.subplots(2, 5, figsize=(14, 8))
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

cmap = plt.cm.viridis
    
# -----Iterate over subregions
o1o2s = snowlines[['O1Region', 'O2Region']].drop_duplicates().dropna().sort_values(by=['O1Region', 'O2Region']).values
ymin, ymax = -0.1, 1.1
for i, (o1, o2) in enumerate(o1o2s):
    # subset snowlines to subregion
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]
    if len(snowlines_subregion) < 1:
        continue
    subregion_name, color = snowlines_subregion[['Subregion', 'color']].values[0]
    if color=='':
        color='k'
    # calculate moving median for all years separately
    yearly_gb = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR']
    yearly = yearly_gb.median().reset_index()
    yearly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    # plot
    for year in yearly['Year'].drop_duplicates().values:
        yearly_year = yearly.loc[yearly['Year']==year]
        ax[i].plot(yearly_year['WOY'], yearly_year['AAR_median'], '-', color=cmap((year-2013)/(2023-2013)), linewidth=1)
    ax[i].set_ylim(ymin, ymax)
    ax[i].grid()
    if (i==0) or (i==5):
        ax[i].set_ylabel('AAR')
    else:
        ax[i].set_yticklabels([])
    if i < 5:
        ax[i].set_xticklabels([])
    N = len(snowlines_subregion['site_name'].drop_duplicates())
    ax[i].set_title(f'{subregion_name} (N={N})')

cbar_ax = fig.add_axes([0.94, 0.4, 0.01, 0.3])
fig.colorbar(matplotlib.cm.ScalarMappable(norm=matplotlib.colors.Normalize(2013, 2023), cmap=cmap),
             cax=cbar_ax, orientation='vertical')
fig.subplots_adjust(hspace=0.2)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_years_stacked.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### Weather data

In [ ]:
# -----Load ERA data
era_fn = os.path.join(scm_path, 'all_ERA_data', 'all_era_data.csv')
era = pd.read_csv(era_fn)
era['Date'] = pd.to_datetime(era['Date'])
# Add Year and WOY columns
era['Year'] = era['Date'].dt.isocalendar().year
era['WOY'] = era['Date'].dt.isocalendar().week

# -----Load AOIs
aois = gpd.read_file(os.path.join(aois_path, aois_fn))
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

# -----Add o1 and o2 region columns
era[['O1Region', 'O2Region']] = '', ''
for site_name in tqdm(era['site_name'].drop_duplicates().values):
    aoi = aois.loc[aois['RGIId']==site_name]
    o1, o2 = aoi[['O1Region', 'O2Region']].values[0]
    era.loc[era['site_name']==site_name, 'O1Region'] = o1
    era.loc[era['site_name']==site_name, 'O2Region'] = o2
era

In [ ]:
# -----Set up figure
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
fig, ax = plt.subplots(10, 2, figsize=(14, 35), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()
ymin1, ymax1 = 0, 1.05
ymin2, ymax2 = 0, 1600
ymin3, ymax3 = 0, 4

weather_columns = ['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']
colors = ['#f4a582', '#2166ac']
q1, q3 = 0.25, 0.75

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

def calculate_weekly_trends(df, column):
    ts_gb = df.groupby(by=['Year', 'WOY'])[column]
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={column: column+'_median'}, inplace=True)
    ts[column+'_Q1'] = ts_gb.quantile(q1).reset_index()[column]
    ts[column+'_Q3'] = ts_gb.quantile(q3).reset_index()[column]
    # calculate moving median for all years stacked
    weekly_gb = df.groupby(by='WOY')[column]
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={column: column+'_median'}, inplace=True)
    weekly[column+'_Q1'] = weekly_gb.quantile(q1).reset_index()[column]
    weekly[column+'_Q3'] = weekly_gb.quantile(q3).reset_index()[column]

    return ts, weekly    

# -----Iterate over subregions
for i, (o1, o2) in enumerate(era[['O1Region', 'O2Region']].drop_duplicates().values):
    
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    print(subregion_name)
    
    # subset snowlines and ERA to subregion
    era_subregion = era.loc[(era['O1Region']==o1) & (era['O2Region']==o2)]
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]

    # calculate moving medians over time and plot
    # PDDs
    column, color = weather_columns[0], colors[0]
    ts, weekly = calculate_weekly_trends(era_subregion, column)
    for year in ts['Year'].drop_duplicates().values + 1:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin1), width=t2-t1, height=ymax1-ymin1, 
                                    facecolor='#d9d9d9', edgecolor='None', zorder=1))
    twin1 = ax[i*2].twinx()
    twin1.fill_between(ts['date'], ts[column+'_median'], np.zeros(len(ts)), 
                       facecolor=color, alpha=0.5, zorder=2)
    twin1.set_ylabel('$\Sigma$PDDs', color=color)
    twin1.spines['left'].set_color(color)
    twin1.tick_params(axis='y', colors=color)
    twin1.set_ylim(ymin2, ymax2)
    # Snowfall
    twin2 = ax[i*2].twinx()
    column, color = weather_columns[1], colors[1]
    ts, weekly = calculate_weekly_trends(era_subregion, column)
    twin2.plot(ts['date'], ts[column+'_median'], '-', color=color, linewidth=2, zorder=3)
    twin2.set_ylabel('$\Sigma$Snowfall [m.w.e.]', color=color)
    twin2.spines.right.set_color(color)
    twin2.spines.right.set_position(("axes", 1.1))
    twin2.tick_params(axis='y', colors=color)
    twin2.set_ylim(ymin3, ymax3)
    # AAR
    column, color = 'AAR', 'k'
    ts, weekly = calculate_weekly_trends(snowlines_subregion, column)
    ax[i*2].set_ylim(ymin1, ymax1)
    ax[i*2].plot(ts['date'], ts[column+'_median'], '.', color=color, markersize=7, zorder=5)
    ax[i*2].set_ylabel('AAR', color=color)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_Q1'], '-', linewidth=1, color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_Q3'], '-', linewidth=1, color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_median'], '-', linewidth=2, color=color)
    ax[(i*2)+1].set_ylim(ymin1, ymax1)
    # adjust axes
    N = len(era_subregion['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{subregion_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[i*2].set_xlim(np.datetime64('2013-01-01'), np.datetime64('2024-02-01'))
    ax[(i*2)+1].grid()

fig.subplots_adjust(hspace=0.25, wspace=0.4)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_pdd_snowfall.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

## Median AARs with different characteristics

In [ ]:
# -----Load minimum snow cover characteristics
min_snow_cover_stats_fn = os.path.join(scm_path, 'results', 'min_snow_cover_stats.csv')
min_snow_cover_stats = pd.read_csv(min_snow_cover_stats_fn)
min_snow_cover_stats['geometry'] = min_snow_cover_stats['geometry'].apply(wkt.loads)
# sort by region numbers
min_snow_cover_stats.sort_values(by=['O1Region', 'O2Region'], inplace=True)
# add cluster column
min_snow_cover_stats[['cluster', 'clustName']] = '', ''
for site_name in min_snow_cover_stats['RGIId'].drop_duplicates().values:
    clusters_site = clusters.loc[clusters['site_name']==site_name]
    min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name, 'cluster'] = clusters_site['cluster'].values[0]
    min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name, 'clustName'] = clusters_site['clustName'].values[0]

# add glacier boundary x and y centroid coordinates for plotting
min_snow_cover_stats['centroid_x'] = [x.centroid.coords.xy[0][0] for x in min_snow_cover_stats['geometry']]
min_snow_cover_stats['centroid_y'] = [x.centroid.coords.xy[1][0] for x in min_snow_cover_stats['geometry']]

# -----Load country outlines for plotting
countries_fn = os.path.join(scm_path, '..', 'GIS_data', 'countries_shp', 'countries.shp')
countries = gpd.read_file(countries_fn)
usca = countries.loc[(countries['NAME']=='United States') | (countries['NAME']=='Canada')].reset_index(drop=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# -----Plot maps of minimum AARs by various columns
columns = ['Subregion', 'clustName', 'Aspect', 'Zmed', 'Area', 'Slope']
columns_display = ['Subregion', 'Climate cluster', 'Aspect [degrees]', 'Median elevation [m]', 'Area [km$^2$]', 'Slope [degrees]']
palettes = [dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values),
            cluster_cmap_dict,
            'hls', 
            'mako', 
            'viridis', 
            'crest']
bins_list = ['N/A', 
             'N/A',
             np.linspace(0,360, num=7), 
             np.linspace(500, 3000, num=6),
             np.linspace(0, 750, num=7),
             np.linspace(0, 35, num=8)]
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
xmin, xmax = -168, -112
ymin, ymax = 45, 65
for column, column_display, palette, bins in zip(columns, columns_display, palettes, bins_list):
    fig, ax = plt.subplots(2, 1, figsize=(10,10), gridspec_kw={'height_ratios':[2.5, 1]})
    # plot country outlines on the map
    usca.plot(ax=ax[0], facecolor='None', edgecolor='grey')
    if column=='Subregion':
        # plot points on the map
        scatter = sns.scatterplot(data=min_snow_cover_stats, x='centroid_x', y='centroid_y', 
                                  hue=column, palette=palette, size='AAR_P50_min', 
                                  sizes=(5, 100), legend=True, ax=ax[0])
        # plot boxplots
        sns.boxplot(data=min_snow_cover_stats, x=column, y='AAR_P50_min', showfliers=False, palette=palette, ax=ax[1])
        ax[1].set_xticks(ax[1].get_xticks())
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
        # calculate number of obserations in each bin
        nobs = min_snow_cover_stats['Subregion'].value_counts().values
    elif column=='clustName':
        # plot points on the map
        scatter = sns.scatterplot(data=min_snow_cover_stats, x='centroid_x', y='centroid_y', 
                                  hue=column, palette=palette, size='AAR_P50_min', 
                                  sizes=(5, 100), legend=True, ax=ax[0])
        # plot boxplots
        sns.boxplot(data=min_snow_cover_stats, x=column, y='AAR_P50_min', showfliers=False, palette=palette, ax=ax[1])
        ax[1].set_xticks(ax[1].get_xticks())
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
        # calculate number of obserations in each bin
        nobs = min_snow_cover_stats['cluster'].value_counts().values
    else:
        # plot points on the map
        sns.scatterplot(data=min_snow_cover_stats, x='centroid_x', y='centroid_y', 
                        hue=column, size='AAR_P50_min', palette=palette,
                        sizes=(5, 100), legend=True, ax=ax[0])
        # add bin column
        min_snow_cover_stats[column + '_bin'] = pd.cut(min_snow_cover_stats[column], bins)
        # plot boxplots
        sns.boxplot(data=min_snow_cover_stats, x=column + '_bin', y='AAR_P50_min', showfliers=False, palette=palette, ax=ax[1])
        # calculate number of observations in each bin
        nobs = min_snow_cover_stats[column + '_bin'].value_counts().values
    # Add number of observations to the boxplot
    nobs = [str(x) for x in nobs.tolist()]
    pos = range(len(nobs))
    for tick in pos:
        ax[1].text(pos[tick], 1.0,
                   nobs[tick],
                   horizontalalignment='center',
                   color='k')
    # adjust legend and axes
    sns.move_legend(ax[0], "center right", bbox_to_anchor=[1.1, 0.4, 0.2, 0.2])
    ax[0].set_xlabel('')
    ax[0].set_ylabel('')
    ax[0].grid()
    ax[0].set_xlim(xmin, xmax)
    ax[0].set_ylim(ymin, ymax)
    ax[1].set_xlabel(column_display)
    ax[1].set_ylim(0, 0.95)
    ax[1].set_ylabel('AAR')

    # Edit legend labels for map
    handles, labels = ax[0].get_legend_handles_labels()
    labels = [x.replace('AAR_P50_min', 'Median AAR').replace(column, column_display) for x in labels]
    ax[0].legend(handles, labels, loc='center right')
    if column=='clustName':
        anchor = [1.15, 0.4, 0.2, 0.2]
    else: 
        anchor = [1.1, 0.4, 0.2, 0.2]
    sns.move_legend(ax[0], "center right", bbox_to_anchor=anchor)
    
    plt.show()
    
    # Save figure
    fig_fn = 'aars_map_' + column + '.png'
    fig.savefig(os.path.join(figures_out_path, fig_fn), dpi=250, bbox_inches='tight')
    print('figure saved to file: ', os.path.join(figures_out_path, fig_fn))

## AAR-PDD linear fits

In [ ]:
# Load AAR-PDD linear fits
aar_pdd_linear_fn = os.path.join(scm_path, 'results', 'aar_pdd_snowfall_linear_fit_sites.csv')
aar_pdd_linear_df = pd.read_csv(aar_pdd_linear_fn)
# Add subregion name and color
aar_pdd_linear_df[['Subregion', 'color']] = '', ''
for o1, o2 in aar_pdd_linear_df[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    aar_pdd_linear_df.loc[(aar_pdd_linear_df['O1Region']==o1) & (aar_pdd_linear_df['O2Region']==o2), 'Subregion'] = subregion_name
    aar_pdd_linear_df.loc[(aar_pdd_linear_df['O1Region']==o1) & (aar_pdd_linear_df['O2Region']==o2), 'color'] = color

aar_pdd_linear_df

In [ ]:
# Plot box plots
fig, ax = plt.subplots(1, 2, figsize=(14,6), sharey=True)
# by climate cluster
sns.boxplot(data=aar_pdd_linear_df, x='clustName', y='coef_linear', ax=ax[0], 
            palette=cluster_cmap_dict, showfliers=False)
# by subregion
sns.boxplot(data=aar_pdd_linear_df, x='Subregion', y='coef_linear', ax=ax[1],
           palette=dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values), showfliers=False)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
ax[0].set_ylabel('AAR vs. PDD linear fit coefficient')
ax[1].set_ylabel('')
ax[0].grid()
ax[1].grid()
fig.subplots_adjust(wspace=0.1)
plt.show()

## Distribution of climates in each subregion

In [ ]:
# Add subregion names to aois
for o1, o2 in aois[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    aois.loc[(aois['O1Region']==o1) & (aois['O2Region']==o2), 'Subregion'] = subregion_name

In [ ]:
# Group AOIs by Subregion and cluster to count values
grouped = aois.groupby(['Subregion', 'clustName']).size().unstack(fill_value=0)
# sort columns and indices
cluster_order = ['Oceanic', 'Continental', 'Transitional-Continental', 'Transitional-Temperate', 'Temperate']
grouped = grouped[cluster_order]
subregions_order = ['Aleutians', 'Alaska Range', 'W. Chugach Mtns.', 
                    'St. Elias Mtns.', 'N. Rockies', 'N. Coast Ranges', 
                    'N. Cascades', 'C. Rockies', 'S. Cascades', 'S. Rockies']
grouped = grouped.reindex(subregions_order)
# subset into roughly coastal and not coastal groups
group1 = ['S. Cascades', 'N. Cascades', 'W. Chugach Mtns.', 'N. Coast Ranges', 'Aleutians']
grouped1 = grouped.loc[group1]
group2 = ['S. Rockies', 'C. Rockies', 'N. Rockies', 'St. Elias Mtns.', 'Alaska Range']
grouped2 = grouped.loc[group2]

# Plot stacked bar plot
fig, ax = plt.subplots(1, 2, figsize=(12,6))
grouped1.plot(kind='barh', stacked=True, legend=True,
              color=[cluster_cmap_dict.get(x, 'black') for x in grouped.columns], 
              ax=ax[0])
grouped2.plot(kind='barh', stacked=True, legend=False,
              color=[cluster_cmap_dict.get(x, 'black') for x in grouped.columns], 
              ax=ax[1])
ax[0].set_xlabel('Counts')
ax[1].set_xlabel('Counts')
ax[1].set_ylabel('')
fig.suptitle('Distribution of clusters within each Subregion')
# adjust legend
handles, labels = ax[0].get_legend_handles_labels()
ax[0].get_legend().remove()
fig.subplots_adjust(wspace=0.35)
fig.legend(handles, labels, title='Cluster', loc='upper right', bbox_to_anchor=[0.93, 0.4, 0.2, 0.2])
plt.show()

# Save to file
fig_fn = os.path.join(figures_out_path, 'subregions_clusters_distribution.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## Plot linear changes in AARs for 2016-2023 by terrain characteristics

In [ ]:
# -----Load AARs linear fit
aars_linear_fit_fn = 'minimum_AARs_linear_fit.csv'
aars_linear_fit = pd.read_csv(os.path.join(snowlines_path, aars_linear_fit_fn))

# -----Add glacier terrain and geometry columns
columns = ['geometry', 'centroid_x', 'centroid_y', 'O1Region', 'O2Region', 'Subregion', 'color', 'Aspect', 'Slope', 'Zmed', 'Area']
for column in columns:
    aars_linear_fit[column] = np.nan
# iterate over site names
for site_name in aars_linear_fit['site_name'].drop_duplicates().values:
    min_snow_cover_stats_site = min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name]
    try:
        for column in columns:
            aars_linear_fit.loc[aars_linear_fit['site_name']==site_name, column] = [min_snow_cover_stats_site[column].values[0]]
    except:
        continue

aars_linear_fit.dropna(inplace=True)

# -----Add column for negative value of change category to show more negative values as larger markers
aars_linear_fit['linear_fit_coef_neg'] = - aars_linear_fit['linear_fit_coef']
# Sort by region numbers
aars_linear_fit.sort_values(by=['O1Region', 'O2Region'], inplace=True)
aars_linear_fit

In [ ]:
# -----Plot map of AARs linear fit with various columns
columns = ['Subregion', 'Aspect', 'Zmed', 'Area', 'Slope']
columns_display = ['Subregion', 'Aspect [degrees]', 'Median elevation [m]', 'Area [km$^2$]', 'Slope [degrees]']
palettes = [dict(aars_linear_fit[['Subregion', 'color']].drop_duplicates().values),
            'hls', 
            'mako', 
            'viridis', 
            'crest']
bins_list = ['N/A', 
             np.linspace(0,360, num=7), 
             np.linspace(500, 3000, num=6),
             np.linspace(0, 750, num=7),
             np.linspace(0, 35, num=8)]
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
xmin, xmax = -168, -112
ymin, ymax = 45, 65
def map_size(value):
    return np.abs(value) * 100
for column, column_display, palette, bins in zip(columns, columns_display, palettes, bins_list):
    fig, ax = plt.subplots(2, 1, figsize=(10,10), gridspec_kw={'height_ratios':[2.5, 1]})
    # plot country outlines on the map
    usca.plot(ax=ax[0], facecolor='None', edgecolor='grey')
    # plot points on the map
    if type(bins) != str:
        sns.scatterplot(data=aars_linear_fit, x='centroid_x', y='centroid_y', 
                        hue=column, palette=palette, size='linear_fit_coef_neg', sizes=(5, 150), legend=True, ax=ax[0])
    else:
        sns.scatterplot(data=aars_linear_fit, x='centroid_x', y='centroid_y', 
                        hue=column, palette=palette, size='linear_fit_coef_neg', sizes=(5, 150), legend=True, ax=ax[0])
    ax[0].set_xlabel('')
    ax[0].set_ylabel('')
    ax[0].grid()
    ax[0].set_xlim(xmin, xmax)
    ax[0].set_ylim(ymin, ymax)
    # plot regional barplots
    if type(bins) != str:
        # add bin column
        aars_linear_fit[column + '_bin'] = pd.cut(aars_linear_fit[column], bins)
        # plot boxplots
        sns.boxplot(data=aars_linear_fit, x=column + '_bin', y='linear_fit_coef', showfliers=False, palette=palette, ax=ax[1])
    else:
        # plot boxplots
        sns.boxplot(data=aars_linear_fit, x=column, y='linear_fit_coef', showfliers=False, palette=palette, ax=ax[1])
    if type(bins) == str:
        ax[1].set_xticks(ax[1].get_xticks())
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel(column_display)
    ax[1].set_ylabel('AAR linear trend [per year]')
    
    # Edit legend labels for map
    handles, labels = ax[0].get_legend_handles_labels()
    labels = [x.replace('linear_fit_coef_neg', 'AAR linear trend [per year]').replace(column, column_display) for x in labels]
    labels = [x.replace('0.', '-0.').replace('−-0', '0') for x in labels]
    ax[0].legend(handles, labels, loc='center right')
    sns.move_legend(ax[0], "center right", bbox_to_anchor=[1.15, 0.4, 0.2, 0.2])

    plt.show()

    # Save figure
    fig_fn = 'AARs_linear_fit_map_' + column + '.png'
    fig.savefig(os.path.join(figures_out_path, fig_fn), dpi=250, bbox_inches='tight')
    print('figure saved to file: ', os.path.join(figures_out_path, fig_fn))

## Plot AAR and linear fit time series for each site

In [ ]:
# Load snowlines
snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))
snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
# Load minimum AAR fits
min_aars_fn = 'minimum_AARs_linear_fit.csv'
min_aars = pd.read_csv(os.path.join(snowlines_path, min_aars_fn))


In [ ]:
# Iterate over sites
for site_name in tqdm(min_aars['site_name'].drop_duplicates().values):
    # Grab snowlines
    snowlines_site = snowlines.loc[snowlines['site_name']==site_name]
    # Grab min AARs
    min_aars_site = min_aars.loc[min_aars['site_name']==site_name].reset_index(drop=True)
    # Plot
    fig, ax = plt.subplots(1,1,figsize=(8,6))
    ax.plot(snowlines_site['datetime'], snowlines_site['AAR'], '.k', markersize=1)
    ax.plot(min_aars_site['minimum_AARs_dts'], min_aars_site['minimum_AARs'], '*m', markersize=2)
    ax.grid()
    ax.set_title(site_name)
    plt.show()

In [ ]:
ast.literal_eval(min_aars['minimum_AARs'][0])#.apply(ast.literal_eval)

## Plot AAR and PDD time series

In [ ]:
# -----Load all snowlines
snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))
snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')

# -----Load all ERA data
eras = pd.read_csv(os.path.join(eras_path, eras_fn))
eras['Date'] = pd.to_datetime(eras['Date'], format='mixed')

# -----Load all AOIs
aois = gpd.read_file(os.path.join(aois_path, aois_fn))
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

In [ ]:
# -----Plot median trends for each subregion
# add year and WOY columns to snowlines and ERA
snowlines['Year'] = snowlines['datetime'].dt.isocalendar().year
snowlines['WOY'] = snowlines['datetime'].dt.isocalendar().week
eras['Year'] = eras['Date'].dt.isocalendar().year
eras['WOY'] = eras['Date'].dt.isocalendar().week
# grab all subregions
subregions = aois[['O1Region', 'O2Region']].drop_duplicates().values
# Set up figure
fig, ax = plt.subplots(len(subregions), 1, figsize=(10, 4*len(subregions)))
text_labels = [x for x in string.ascii_lowercase[0:len(subregions)]]
# Iterate over subregions
for i, (o1region, o2region) in enumerate(subregions):
    
    subregion_name, color = f.determine_subregion_name_color(o1region, o2region)
    
    # Grab all AOIs in subregion
    aois_subregion = aois.loc[(aois['O1Region']==o1region) & (aois['O2Region']==o2region)]
    site_names_subregion = aois_subregion['RGIId'].drop_duplicates().values
    # Grab all snowlines in subregion
    Isubregion = [i for i in np.arange(len(snowlines)) 
                  if snowlines.loc[i, 'site_name'] in site_names_subregion]
    snowlines_subregion = snowlines.iloc[Isubregion]
    # Calculate AAR median and IQR trends over time
    aar_median = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR'].median().reset_index()
    aar_median['deciyear'] = aar_median['Year'] + (aar_median['WOY'] / 52)
    # aar_q1 = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR'].apply(np.quantile(q=0.25))
    # aar_q3 = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR'].apply(np.quantile(q=0.75))
    # Grab all ERA data in subregion
    Isubregion = [i for i in np.arange(len(eras)) 
                  if eras.loc[i, 'site_name'] in site_names_subregion]
    eras_subregion = eras.iloc[Isubregion]
    # Calculate PDD median and IQR trends over time
    pdd_median = eras_subregion.groupby(by=['Year', 'WOY'])['Cumulative_Positive_Degree_Days'].median().reset_index()
    pdd_median['deciyear'] = pdd_median['Year'] + (pdd_median['WOY'] / 52)
    # pdd_q1 = eras_subregion.groupby(by=['Year', 'WOY'])['Cumulative_Positive_Degree_Days'].apply(np.quantile(q=0.25))
    # pdd_q3 = eras_subregion.groupby(by=['Year', 'WOY'])['Cumulative_Positive_Degree_Days'].apply(np.quantile(q=0.75))

    # Plot
    ax[i].plot(aar_median['deciyear'], aar_median['AAR'], '.-b', linewidth=0.5, markersize=3)
    ax[i].grid()
    ax[i].set_ylim(0, 1)
    ax[i].set_ylabel('Transient AAR', color='b')
    ax[i].tick_params(axis='y', colors='b')
    ax[i].set_title(text_labels[i] + ') ' + subregion_name)
    ax2 = ax[i].twinx()
    ax2.plot(pdd_median['deciyear'], pdd_median['Cumulative_Positive_Degree_Days'], '-m')
    ax2.set_ylabel('$\Sigma$PDD', color='m')
    ax2.tick_params(axis='y', colors='m')
    
plt.show()

# -----Save figure
fig_fn = 'time_series_AAR_PDDs.png'
fig.savefig(os.path.join(figures_out_path, fig_fn), dpi=200, bbox_inches='tight')
print('figure saved to file:', os.path.join(figures_out_path, fig_fn))


## Plot weather data time series

## Plot correlation statistics

### AAR-PDD correlations

In [ ]:
# -----Load AAR-PDD correlations for each subregion
corr_coeffs_fns = sorted(glob.glob(os.path.join(eras_path, 'correlation_*aar-pdd*.csv')))
corr_coeffs = pd.DataFrame()
for corr_coeffs_fn in corr_coeffs_fns:
    # grab subregion name from file name
    subregion_name = os.path.basename(corr_coeffs_fn).split('aar-pdd_')[1].split('.csv')[0]
    # load correlation coefficients for all sites
    corr_coeffs_subregion = pd.read_csv(corr_coeffs_fn)
    # add subregion name to dataframe
    corr_coeffs_subregion['Subregion'] = subregion_name    
    # concatenate to full dataframe
    corr_coeffs = pd.concat([corr_coeffs, corr_coeffs_subregion])
corr_coeffs

In [ ]:
# -----Plot boxplots
palette = dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values)
fig, ax = plt.subplots(figsize=(8,6))
sns.boxplot(data=corr_coeffs, x='Subregion', y='AAR-PDD Corr. Coeff.', palette=palette, ax=ax)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('')
ax.set_ylabel('AAR - $\Sigma$PDD Correlation Coefficients')
# ax.grid()
plt.show()

# -----Save to file
fig_fn = os.path.join(figures_out_path, 'aar-pdd_correlation_coefficients_boxplot.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file: ', fig_fn)

## Plot scatter plots of AAR vs. cumulative PDDs

In [ ]:
# -----Load all snowlines
snowlines = pd.read_csv(os.path.join(snowlines_path, snowlines_fn))
snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
snowlines['Date'] = snowlines['datetime'].values.astype('datetime64[D]')

# -----Load all ERA data
eras = pd.read_csv(os.path.join(eras_path, eras_fn))
eras['Date'] = pd.to_datetime(eras['Date'], format='mixed')

# -----Load all AOIs
aois = gpd.read_file(os.path.join(aois_path, aois_fn))
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

In [ ]:
# -----Fit multiple linear regression model to AAR, PDDs, and Snowfall
# intialize full dataframe
snowlines_eras_merged = pd.DataFrame()
# iterate over sites
for site_name in tqdm(snowlines['site_name'].drop_duplicates().values):
    # subset dataframes to site
    snowlines_site = snowlines.loc[snowlines['site_name']==site_name]
    eras_site = eras.loc[eras['site_name']==site_name]
    # merge snowlines and ERA dataframes
    snowlines_eras_merged_site = snowlines_site.merge(eras_site, how='left', on='Date')
    snowlines_eras_merged_site.rename(columns={'site_name_x': 'site_name'}, inplace=True)
    # save in dataframe
    snowlines_eras_merged_site = snowlines_eras_merged_site[['site_name', 
                                                             'AAR', 
                                                             'Cumulative_Positive_Degree_Days',
                                                             'Cumulative_Precipitation_mwe',
                                                             'Cumulative_Snowfall_mwe']]
    snowlines_eras_merged_site.reset_index(drop=True, inplace=True)
    snowlines_eras_merged_site.dropna()
    # plot
    ax.plot(snowlines_eras_merged_site['Cumulative_Positive_Degree_Days'], 
            snowlines_eras_merged_site['AAR'], 
            '.', markersize=1)
    # concatenate to full dataframe
    snowlines_eras_merged = pd.concat([snowlines_eras_merged, snowlines_eras_merged_site])
    

In [ ]:
# -----Fit multi-linear regression model
snowlines_eras_merged.dropna(inplace=True)
model = LinearRegression()
X_columns = ['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']
# fit model
X = snowlines_eras_merged[X_columns]
y = snowlines_eras_merged['AAR'].values
model_fit = model.fit(X, y)
# calculate R^2
r2 = model_fit.score(X, y)
# plot
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')
ax.plot(X[X_columns[0]], X[X_columns[1]], y, '.', color='grey', markersize=0.5)
y_pred = model_fit.predict(X)
ax.plot(X[X_columns[0]], X[X_columns[1]], y_pred, '.k')
ax.set_xlabel(X_columns[0])
ax.set_ylabel(X_columns[1])
ax.set_zlabel('AAR')
ax.set_title('AAR = ' + str(np.round(model_fit.coef_[0], 5)) + '*$\Sigma$PDDs ' 
             + str(np.round(model_fit.coef_[1], 5)) + '*$\Sigma$Snowfall' 
             + '\nR$^2$ = ' + str(np.round(r2,55)))
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'aar-pdd-snowfall_multiple_linear_regression.png')
fig.savefig(fig_fn, dpi=200, bbox_inches='tight')
print('figure saved to file:', fig_fn)

## Plot machine learning results

### Feature importances

## Climate/weather sensitivities

In [ ]:
# -----Define dictionary for colors of each subregion
color_dict = {
    'Brooks Range': 'c',
    'Alaska Range': '#1f78b4',
    'Aleutians': '#6d9c43',
    'W. Chugach Mtns.': '#264708',
    'St. Elias Mtns.': '#fb9a99',
    'N. Coast Ranges': '#e31a1c',
    'N. Rockies': '#cab2d6',
    'N. Cascades': '#fdbf6f',
    'C. Rockies': '#9657d9',
    'S. Cascades': '#ff7f00',
    'S. Rockies': '#6a3d9a'
}

In [ ]:
ml_path = os.path.join(scm_path, 'machine_learning')
sens_run_fns = sorted(glob.glob(os.path.join(ml_path, 'aar_sensitivity_tests*.csv')))

# -----Define regression model
model = LinearRegression()

# -----Set up figure
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
fig, ax = plt.subplots(2, 2, figsize=(12, 12), gridspec_kw={'width_ratios': [3, 1]})
ax = ax.flatten()

# -----Iterate over PDDs and Snowfall
for i, column in enumerate(['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']):
    # Grab file names for column
    sens_run_column_fns = [x for x in sens_run_fns if column in x]
    # Iterate over file names
    for j, fn in enumerate(sens_run_column_fns):
        # grab subregion name from file name
        subregion_name = fn.split('aar_sensitivity_tests_')[1].split('_' + column)[0]
        color = color_dict[subregion_name]
        # load file
        sens_run = pd.read_csv(fn)
        # remove mean conditions row
        sens_run = sens_run.loc[sens_run['scenario']!='mean']
        # fit linear regression to cumsum(PDDs) and AAR
        model_fit = model.fit(sens_run[column].values.reshape(-1, 1), 
                              sens_run['AAR'].values.reshape(-1, 1))
        # grab linear fit coefficient
        coef = model_fit.coef_[0][0]
        # add to DataFrame
        fit_df = pd.DataFrame({'Subregion': [subregion_name],
                               'Color': color,
                               'Linear_Fit_Coefficient': [coef]})
        fits_df = pd.concat([fits_df, fit_df])
        # plot
        lines = ax[i*2].plot(sens_run[column], 
                             sens_run['AAR'], '-', label=subregion_name, color=color)
        ax[(i*2)+1].plot(j, coef, 'o', color=color, markersize=5, label=subregion_name)
    ax[i*2].set_ylabel('AAR')
    ax[i*2].set_xlabel(column)
    ax[i*2].grid()
    ax[(i*2)+1].set_ylabel('Linear fit coefficient')
    ax[(i*2)+1].grid()

leg = ax[1].legend(loc='right', bbox_to_anchor=[1.9, -0.2, 0.2, 0.2])
fig.subplots_adjust(wspace=0.3)
plt.show()


In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
model = BaggingRegressor()
model_fit = model.fit(sens_run[column].values.reshape(-1, 1), sens_run['AAR'].values.reshape(-1, 1))
model_fit